In [1]:
import pandas as pd


In [2]:
train_path = '../datas/train.csv'
test_path = '../datas/test.csv'
sample_path = '../datas/sample_submission.csv'
mapping_path = '../datas/misconception_mapping.csv'

m_map = pd.read_csv(mapping_path)
mis_concepts = m_map['MisconceptionName'].tolist()
m_map.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


In [3]:
train_df = pd.read_json('../datas/data/train_data.json')
train_sub_df = pd.read_csv("./datas/nvembed_v1/nvembed_no_llm_train.csv")
train_df = train_df.merge(train_sub_df, on='QuestionId_Answer', how='left')
train_df.head()

,QuestionId_Answer,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,InCorrectAnswer,InCorrectAnswerText,MisconceptionId_x,MisconceptionName,MisconceptionId_y
0,0_D,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,A,\( 3 \times(2+4)-5 \),D,Does not need brackets,1672,"Confuses the order of operations, believes add...",1005 1507 2532 315 706 1672 1941 1345 2306 139...
1,1_A,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,A,\( m+1 \),2142,Does not know that to factorise a quadratic ex...,1593 2398 1755 167 3 891 59 363 265 1766 2549 ...
2,1_B,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,B,\( m+2 \),143,Thinks that when you cancel identical terms fr...,1593 2398 1755 167 3 891 1766 363 59 265 2549 ...
3,1_C,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",D,Does not simplify,C,\( m-1 \),2142,Does not know that to factorise a quadratic ex...,1593 2398 1755 891 59 167 3 1766 363 2549 1540...
4,2_A,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,Tom and Katie are discussing the \( 5 \) plant...,B,Only\nKatie,A,Only\nTom,1287,Believes if you changed all values by the same...,1073 1287 2439 397 2471 1677 1059 691 1349 231...


In [4]:
def get_random_neg(label, preds, k=5):
    preds_list = preds.split()
    label = int(label)
    
    samples = []
    for neg in preds_list:
        neg = int(neg)
        if neg == label:continue
        samples.append(mis_concepts[neg])
    return samples[0:k]



In [5]:
total_examples = []
for i in range(len(train_df)):
    tmp = {}
    tmp['query'] = f"ConstructName:{train_df.loc[i, 'ConstructName']}\nQuestionText:{train_df.loc[i, 'QuestionText']}\nCorrectAnswerText:{train_df.loc[i, 'CorrectAnswerText']}\nInCorrectAnswerText:{train_df.loc[i, 'InCorrectAnswerText']}"
    tmp['pos'] = [train_df.loc[i, 'MisconceptionName']]
    tmp['neg'] = get_random_neg(int(train_df.loc[i, "MisconceptionId_x"]), train_df.loc[i, "MisconceptionId_y"], 15)
    tmp['prompt'] = "Given a mathematical question that includes both a correct and an incorrect answer, along with a specific misconception, determine whether this misconception can explain why the student selected the incorrect answer. Please provide a prediction of either 'Yes' or 'No'."
    total_examples.append(tmp)
s = set()
for k in total_examples:
    s |= set(k['neg'])

# assert len(s) == len(mis_concepts)

In [6]:
import json
filename = './datas/nvembed_v1/nvembed_v1_no_llm_recall_top_15_for_rank.jsonl'
with open(filename, 'w', encoding='utf-8') as f:
    for entry in total_examples:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')  

In [10]:
total_examples[0]

{'query': 'ConstructName:Use the order of operations to carry out calculations involving powers\nQuestionText:\\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?\nCorrectAnswerText:\\( 3 \\times(2+4)-5 \\)\nInCorrectAnswerText:Does not need brackets',
 'pos': ['Confuses the order of operations, believes addition comes before multiplication '],
 'neg': ['Carries out operations from left to right regardless of priority order, unless brackets are used',
  'Carries out operations from left to right regardless of priority order',
  'Believes order of operations does not affect the answer to a calculation',
  'Has not realised that the answer may be changed by the insertion of brackets',
  'Carries out operations from right to left regardless of priority order',
  'Confuses the order of operations, believes subtraction comes before multiplication ',
  'Inserts brackets but not changed order of operation',
  'Applies BIDMAS in strict order (does n

In [18]:
len(mis_concepts)

2587

In [17]:
len(s)

2562